In [1]:
import subprocess

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


/Users/marumarukun/Documents/my_study/whisper/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
YOUTUBE_ID = "wxvxP8yHEPg"  # https://www.youtube.com/watch?v=wxvxP8yHEPg の末尾のID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"


def dl_yt(yt_url):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True, check=False)


dl_yt(f"https://youtu.be/{YOUTUBE_ID}")


[youtube] Extracting URL: https://youtu.be/wxvxP8yHEPg
[youtube] wxvxP8yHEPg: Downloading webpage
[youtube] wxvxP8yHEPg: Downloading ios player API JSON
[youtube] wxvxP8yHEPg: Downloading web creator player API JSON
[youtube] wxvxP8yHEPg: Downloading m3u8 information
[info] wxvxP8yHEPg: Downloading 1 format(s): 251
[download] wxvxP8yHEPg.mp3 has already been downloaded
[ExtractAudio] Not converting audio wxvxP8yHEPg.mp3; file is already in target format mp3


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


In [4]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)


In [5]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


In [10]:
result = pipe(AUDIO_FILE_NAME, return_timestamps="word")


/Users/marumarukun/Documents/my_study/whisper/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
WhisperModel is using WhisperSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [12]:
for chunk in result["chunks"]:
    print(f"[{chunk['timestamp'][0]:.2f}s - {chunk['timestamp'][1]:.2f}s] {chunk['text']}")


[3.12s - 3.12s] ん
[3.12s - 6.06s] 監督よりファンの皆様へお挨拶申し上げます
[6.06s - 8.18s]  新庄監督お願いいたします
[12.78s - 13.26s] なんと
[13.26s - 15.12s]  皆さんのおかげで
[15.94s - 18.56s] 2017年7年ぶりに
[18.56s - 22.76s]  200人200万人突破できることができました
[23.58s - 24.00s] ありがとうございます
[31.48s - 34.04s] 3年間成長した選手たちに
[34.78s - 36.98s] 目一杯の拍手をお願いします
[46.66s - 49.30s] 皆さんに一つお願いがあります
[50.92s - 52.48s] 来年
[54.78s - 57.14s] レイエスとマルチネス
[58.24s - 61.60s] ファイターズのユニフォームを着てほしくないですか
[66.58s - 70.08s] よそのチームのユニフォームは似合わないですよね
[73.84s - 77.34s] 皆さんどこも行かないで運動を始めてください
[82.76s - 87.40s] 僕たちは日本一しか今考えていません
[89.36s - 92.56s] ドラマ大公開ここからが一番楽しいです
[93.40s - 95.18s] 皆さん僕についてきてください
[96.78s - 97.88s] 1年間ありがとうございました
[103.08s - 103.52s] ありがとうございました
[137.28s - 137.28s] ご視聴ありがとうございました


In [23]:
# 複数の音声ファイルを処理する場合は、リストを渡す
results = pipe(["../data/input/g_02.mp3", "../data/input/g_22.mp3"], return_timestamps=True, batch_size=2)


/Users/marumarukun/Documents/my_study/whisper/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [30]:
for result in results:
    for chunk in result["chunks"]:
        print(f"[{chunk['timestamp'][0]:.2f}s - {chunk['timestamp'][1]:.2f}s] {chunk['text']}")
    print("\n")


[0.00s - 8.34s] F博士が発見開発した乳酸菌 ヨーグルトや甘いドリンクなんかに入っているものと同じなんですか?
[11.06s - 13.00s] 何ですか?この写真は
[13.00s - 16.94s] いろんな乳酸菌の働きを比較した表ですね
[16.94s - 21.70s] うわー、白玉菌っていてほしくないやつですね


[0.00s - 10.84s] 血管収縮には血管閉合筋細胞内のカルシウム濃度が関与しており 血管閉合筋細胞内へのカルシウム流入をブロックすれば血管収縮を抑制できます
[10.84s - 20.62s] しかし近年カルシウム流入とは独立した経路としてロックが見出され 血管収縮に深く関与していることが明らかになりました


